# Lab 5: RNAfold/plot and Using the Subprocess Module

## Grab Sequences from Penchovsky and Breaker 

Please copy down the sequences for the YES-1 riboswitch from Figure 2a, the NOT-1 riboswitch in Figure 4a, the AND-1riboswitch in Figure 5a, and the OR-1 riboswitch in Figure 6a.For each riboswitch, produce a table in your iPython notebook that contains the following information: 
 1. riboswitch name 
 2. start and end coordinates of OBS-1 (blue region) 
 3. start and end coordinates of OBS-2 (blue region, only applicable to AND-1 and OR-1) 
 4. start and end coordinates of the two red regions


YES-1: (Figure 2a)
seq:GGGCGACCCUGAUGAGCUUGAGUUUAGCUCGUCACUGUCCAGGUUCAAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCC 
OBS: 26-47
Red regions: 16-21, 49-54

NOT-1: (Figure 4a)
seq:GGCAGGUACAUACAGCUGAUGAGUCCCAAAUAGGACGAAACGCGACACACACCACUAAACCGUGCAGUGUUUUGCGUCCUGUAUUCCACUGC
OBS: 44-66
Red regions: 40-43, 74-77

AND-1: (Figure 5a)
seq:GGGCGACCCUGAUGAGCUUGGUUUAGUAUUUACAGCUCCAUACAUGAGGUGUUAUCCCUAUGCAAGUUCGAUCAGGC GAAACGGUGAAAGCCGUAGGUUGCCCAGAGACAAU
OBS1: 30-45
OBS2: 49-64
Red regions: 16-23, 70-77

OR-1: (Figure 6a)
seq:GGGCGACCCCUGAUGGCUUGGUUGAGUAUUUACAGCUCCAUAUACAUGAGGUGUUCUCCCUACGCAAGUUCGAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCC
OBS1: 26-4 
OBS2: 47-66
Red regions: 16-26, 67-77

## Now, write a routine to fold RNA: 

### Check that RNAfold works in Terminal
Trying it out in Terminal:

    be131-04@ip-172-30-0-167:~$ cd Lab_5
    be131-04@ip-172-30-0-167:~/Lab_5$ RNAfold

Terminal Returns: 
Input string (upper or lower case); @ to quit
....,....1....,....2....,....3....,....4....,....5....,....6....,....7....,....8

Pasted in the following sequence below the numbers: GGGCGACCCUGAUGAGCUUGAGUUUAGCUCGUCACUGUCCAGGUUCAAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCC@ - the at-sign (@) is needed to tell the software to stop

Terminal Returns: 
GGGCGACCCUGAUGAGCUUGAGUUUAGCUCGUCACUGUCCAGGUUCAAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCC@
((((((((.(((((((((.......))))))))).((.((.(((...))).))))..(((((....))))).)))))))).
 minimum free energy = -33.80 kcal/mol

Input string (upper or lower case); @ to quit
....,....1....,....2....,....3....,....4....,....5....,....6....,....7....,....8

I fold a file called rna.ps in my Lab_5 directory

### Run RNAfold commands from within Python

In [12]:
Y1_seq = "GGGCGACCCUGAUGAGCUUGAGUUUAGCUCGUCACUGUCCAGGUUCAAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCC@"

import subprocess

p = subprocess.run('RNAfold', 
                   input= bytes(Y1_seq, 'ascii'),
                   stdout= subprocess.PIPE,
                   stderr= subprocess.PIPE)

#print(p.stderr.decode())
#print(p.stdout.decode())

print(p.stderr.decode()) = nothing

print(p.stdout.decode())=GGGCGACCCUGAUGAGCUUGAGUUUAGCUCGUCACUGUCCAGGUUCAAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCC@
((((((((.(((((((((.......))))))))).((.((.(((...))).))))..(((((....))))).)))))))). (-33.80)

In [33]:
rnafile = open('rna.ps')
rnar= rnafile.read()
foldrna= "GGGCGACCCUGAUGAGCUUGAGUUUAGCUCGUCACUGUCCAGGUUCAAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCC@/n((((((((.(((((((((.......))))))))).((.((.(((...))).))))..(((((....))))).)))))))). (-33.80)"
p = subprocess.run('RNAplot', 
                   input= bytes(foldrna, 'ascii'),
                   stdout= subprocess.PIPE,
                   stderr= subprocess.PIPE)
print(p.stderr.decode())
print(p.stdout.decode())

ERROR: structure missing for record 0





UsageError: Line magic function `%show` not found.


In [43]:
file='Y1_0001_ss.ps'

import os
import glob
for f in glob.glob('Y1_0001_ss.ps'):
    os.system('ps2pdf '+f+' '+f[0:-3]+'.pdf')


Y1_0001_ss.pdf


[Yes gate]!(https://bio.nixt.org:8000/user/be131-04/files/Lab_5/Y1_0001_ss.pdf)


## The meat of the assignment:

Once you have confirmed that your RNAfold routine works and is generating images (check for rna.ps in yourcurrent working directory), begin folding the riboswitches.
1. Start by generating one plot per riboswitch (YES-1, NOT-1, AND-1, and OR-1) with default parameters. Compare each plot to the native conformation given in the publication. 
     Are they the same? Are they different? Are there any stem-loop structures that don’t match up?Whatmight explain the differences? See if you can track down the parameters the authors used and compare them to the default RNAfold parameters (e.g., temperature, algorithm, etc).Don’t kill yourself comparing every last base position to the paper, but do give us a general idea about the presence of stem-loops and whether the red regions have bound each other.
2. Next, simulate OBS binding in YES-1 and NOT-1. The idea here is that we’d like to see what conformation these riboswitches adopt when their target OBS is bound.To simulate OBS binding, we tell RNAfold to prevent the blue OBS region (that we noted down in our table above) from pairing to any other region of the riboswitch.To do this, we pass the –C parameter to RNAfold using a file which you will have to generate.

        RNAfold –C YES-1_contraints.txt
    
    The YES-1_contraints.txtfile should containone letter for each base inYES-1. Use a dot (“.”) for every unconstrained base, and a lower-case xfor every base that you want to prevent from pairing. That is to say, put an x at every base spanning the blue OBS region.Now, fold YES-1 again with constraints and see what you get. Does it look like the self-cleaving form of YES-1 in Figure 2? Are the red regions bound to each other? Do the same for NOT-1.
3. Simulating OBS binding for AND-1 and OR-1 is more complicated. Recall that AND and OR logic gates have two inputs, and so their truth tables have four rows: F/F, T/F, F/T, and T/T.In part (1) we simulated the F/F case, and so now mustsimulate the other three.This will require you to produce three constraintfiles for AND-1, and three for OR-1.One file will have OBS-1 x’d out, another will have OBS-2 x’d out, and the third will have both OBS-1 and OBS-2x’d out.
    Produce the constraint files, run RNAfold, and compare the output files to the paper.Determine whether the red regions (which you noted in your table above) pair with each other for each of the four possible input combinations. Generate a truth table for the AND-1 and the OR-1 riboswitches based on your results. 

According to your results, do the AND-1 and OR-1 riboswitches work as the paperclaims?